In [4]:
from transformers import SiglipImageProcessor, SiglipVisionModel,SiglipVisionConfig
from PIL import Image
# vision_tower=SiglipVisionModel.from_pretrained('google/siglip-base-patch16-224',cache_dir='/public_data/jihai/tmp/siglip-base-patch16-224')
# print(vision_tower.config.hidden_size)
processor=SiglipImageProcessor.from_pretrained('google/siglip-base-patch16-224',cache_dir='/public_data/jihai/tmp/siglip-base-patch16-224')
print(processor.image_mean)
image = Image.open('/public_data/jihai/data/multimodalout/smart_watch_image_test/0.png').convert('RGB')
image = processor.preprocess(image, return_tensors='pt')['pixel_values'][0]
print(image.shape)

/home/jihai/anaconda3/envs/multimodal/lib/python3.9/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[0.5, 0.5, 0.5]
torch.Size([3, 224, 224])


In [5]:
from torchvision import transforms
class VQImageProcessor:
    def __init__(self, input_size):
        self.transform = transforms.Compose([
            transforms.Resize((input_size, input_size)),  # 调整尺寸
            transforms.ToTensor()                        # 转换为Tensor并自动归一化到[0,1
        ])

    def preprocess(self, image,return_tensors='pt'):
        image = self.transform(image)
        image=2.0*image-1.0
        # 添加批次维度并返回
        image=image.unsqueeze(0)
        out={'pixel_values':image}
        return out
processor=VQImageProcessor(256)
image = Image.open('/public_data/jihai/data/multimodalout/smart_watch_image_test/0.png').convert('RGB')
image = processor.preprocess(image, return_tensors='pt')['pixel_values'][0]
print(image.shape)

torch.Size([3, 256, 256])


In [1]:
from vq_model import VQ_models

In [1]:
import torch
import torch.nn as nn
torch.manual_seed(42)
strength=1.1

vision_tower_hidden_dim=768
vision_tower_name='siglip_'

Q=torch.eye(vision_tower_hidden_dim)
torch.save(Q, vision_tower_name+str(strength)+'identity_matrix.pt')

# 设置缩放因子范围（可以根据需要调整）
scale_min, scale_max = 1/strength, 1*strength
shear_std=1*strength
# 生成两个随机缩放因子
scale_factors = torch.rand(vision_tower_hidden_dim) * (scale_max - scale_min) + scale_min  # 范围 [scale_min, scale_max]
# 创建一个对角矩阵，缩放因子在对角线上
scale_matrix = torch.diag(scale_factors)
shear = torch.eye(vision_tower_hidden_dim)
# 生成上三角部分的随机值（非对角线元素）
upper_triangle = torch.triu(
    torch.rand(vision_tower_hidden_dim, vision_tower_hidden_dim) * shear_std,
    diagonal=1  # 仅填充严格上三角部分
)
shear = shear + upper_triangle

# 3. 组合缩放和剪切（顺序：先缩放，后剪切）
#scale_matrix = shear @ scale_matrix
print(scale_matrix)
torch.save(scale_matrix, vision_tower_name+str(strength)+'scale_matrix.pt')

random_matrix = torch.randn(vision_tower_hidden_dim, vision_tower_hidden_dim)
# QR 分解得到正交矩阵 Q
rotation_matrix, _ = torch.linalg.qr(random_matrix, mode='complete')
torch.save(rotation_matrix, vision_tower_name+str(strength)+'rotation_matrix.pt')

translation_matrix=torch.rand(vision_tower_hidden_dim)*0.1*strength
torch.save(translation_matrix, vision_tower_name+str(strength)+'translation_matrix.pt')



tensor([[1.0775, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0838, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.9822,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 1.0584, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0299, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.9151]])


In [2]:

import torch
import torch.nn as nn

class Affine(nn.Module):
    def __init__(self, dim,strength,vision_tower_name):
        super(Affine, self).__init__()
        
        # 生成随机正交矩阵 A（使用 QR 分解）
        #random_matrix = torch.randn(dim, dim)
        # QR 分解得到正交矩阵 Q
        #Q, _ = torch.linalg.qr(random_matrix, mode='complete')  # 使用 torch.linalg.qr
        Q= torch.load(f"{vision_tower_name}_{strength}rotation_matrix.pt")
        S=torch.load(f"{vision_tower_name}_{strength}scale_matrix.pt")
        Q=torch.matmul(Q,S)
        B=torch.load(f"{vision_tower_name}_{strength}translation_matrix.pt")
        # 将 A 和 b 作为常量属性存储，并确保它们不会被训练
        self.register_buffer('A', Q)  # register_buffer 确保 A 不可训练
        self.register_buffer('b', B)  # 随机平移向量 b，不可训练

    def forward(self, x, *args, **kwargs):
        # 仿射变换 y = A * x + b
        x=x.view(-1, self.A.shape[1])
        y=torch.matmul(x,self.A) + self.b
        y=y.view(-1, 6, self.A.shape[0])
        return y

In [3]:
strength=1.1
vision_tower_name='siglip'
transformation= Affine(768, strength, vision_tower_name)

torch.save(transformation.state_dict(), f"{vision_tower_name}_affine_{strength}.pth")

In [1]:

import torch
import torch.nn as nn

class Affine(nn.Module):
    def __init__(self, dim,strength,vision_tower_name):
        super(Affine, self).__init__()
        
        # 生成随机正交矩阵 A（使用 QR 分解）
        #random_matrix = torch.randn(dim, dim)
        # QR 分解得到正交矩阵 Q
        #Q, _ = torch.linalg.qr(random_matrix, mode='complete')  # 使用 torch.linalg.qr
        # Q= torch.load(f"{vision_tower_name}_{strength}rotation_matrix.pt")
        # S=torch.load(f"{vision_tower_name}_{strength}scale_matrix.pt")
        # Q=torch.matmul(Q,S)
        # B=torch.load(f"{vision_tower_name}_{strength}translation_matrix.pt")
        B=torch.zeros(dim)
        Q= torch.load(f"{vision_tower_name}_{strength}rotation_matrix.pt")
        #Q=torch.eye(dim)
        print(Q)
        # 将 A 和 b 作为常量属性存储，并确保它们不会被训练
        self.register_buffer('A', Q)  # register_buffer 确保 A 不可训练
        self.register_buffer('b', B)  # 随机平移向量 b，不可训练

    def forward(self, x, *args, **kwargs):
        # 仿射变换 y = A * x + b
        x=x.view(-1, self.A.shape[1])
        y=torch.matmul(x,self.A) + self.b
        y=y.view(-1, 6, self.A.shape[0])
        return y
    
strength=1
vision_tower_name='siglip'
transformation= Affine(768, strength, vision_tower_name)

torch.save(transformation.state_dict(), f"{vision_tower_name}_affine_rota.pth")

tensor([[-0.0198, -0.0173, -0.0445,  ...,  0.0265, -0.0079,  0.0561],
        [ 0.0374, -0.0910,  0.0089,  ...,  0.0167, -0.0290,  0.0090],
        [ 0.0253,  0.0264, -0.0415,  ...,  0.1131, -0.0368, -0.0104],
        ...,
        [-0.0024, -0.0130, -0.0031,  ...,  0.0099, -0.0239, -0.0692],
        [ 0.0059, -0.0197,  0.0795,  ..., -0.0227,  0.0086,  0.0752],
        [-0.0174, -0.0026,  0.0415,  ...,  0.0335,  0.0274,  0.0208]])
